https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection

In [11]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from model import SSD300, MultiBoxLoss
#from datasets import PascalVOCDataset
from utils import *


from torch.utils.data import Dataset
import json
import os
from PIL import Image
from utils import transform
import pickle

In [13]:
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    boxes = list()
    labels = list()
    difficulties = list()
    for object in root.iter('object'):

        difficult = int(object.find('xml version') == '1')

        label = object.find('name').text.lower().strip()
        if label not in label_map:
            continue

        bbox = object.find('bndbox')
        xmin = int(bbox.find('xmin').text) - 1
        ymin = int(bbox.find('ymin').text) - 1
        xmax = int(bbox.find('xmax').text) - 1
        ymax = int(bbox.find('ymax').text) - 1

        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label_map[label])
        difficulties.append(difficult)

    return {'boxes': boxes, 'labels': labels, 'difficulties': difficulties}

In [14]:
# output_folder = './train_images'
# create_data_lists(output_folder)

In [15]:
# ids= os.listdir('../train_cdc/train_images')
# ids[0].split('.',1)[0]

In [16]:
from sklearn.model_selection import train_test_split

In [18]:
def create_data_lists(output_folder):
    """
    Create lists of images, the bounding boxes and labels of the objects in these images, and save these to file.
    :param voc07_path: path to the 'VOC2007' folder
    :param voc12_path: path to the 'VOC2012' folder
    :param output_folder: folder where the JSONs must be saved
    """
    
    path='../train_cdc/'
    
    train_images = list()
    train_objects = list()
    n_objects = 0

    # Training data

    # Find IDs of images in training data
#     with open(os.path.join(path, 'ImageSets/Main/trainval.txt')) as f:
#         ids = f.read().splitlines()
    ids= os.listdir('../train_cdc/train_images')
    for id in ids:
        
        id = id.split('.',1)[0]
        # Parse annotation's XML file
        objects = parse_annotation(os.path.join(path, 'train_annotations', id + '.xml'))
        if len(objects['boxes']) == 0:
            continue
        n_objects += len(objects)
        train_objects.append(objects)
        train_images.append(os.path.join(path, 'train_images', id + '.jpg'))

    assert len(train_objects) == len(train_images)
    train_images, test_images, train_objects, test_objects = train_test_split(train_images, train_objects, test_size=0.1, random_state=0)
    print(len(train_images),len(test_images),len(train_objects),len(test_objects))
    print(test_images[0:2],test_objects[0:2])
    # Save to file
    with open(os.path.join(output_folder, 'TRAIN_images.json'), 'w') as j:
        json.dump(train_images, j)
    with open(os.path.join(output_folder, 'TRAIN_objects.json'), 'w') as j:
        json.dump(train_objects, j)
        
    with open(os.path.join(output_folder, 'TEST_images.json'), 'w') as j:
        json.dump(test_images, j)
    with open(os.path.join(output_folder, 'TEST_objects.json'), 'w') as j:
        json.dump(test_objects, j)
    with open(os.path.join(output_folder, 'label_map.json'), 'w') as j:
        json.dump(label_map, j)  # save label map too

    print('\nThere are %d training images containing a total of %d objects. Files have been saved to %s.' % (
        len(train_images), n_objects, os.path.abspath(output_folder)))

#     # Test data
#     test_images = list()
#     test_objects = list()
#     n_objects = 0

    # Find IDs of images in the test data
#     with open(os.path.join(voc07_path, 'ImageSets/Main/test.txt')) as f:
#         ids = f.read().splitlines()

#     for id in ids:
#         # Parse annotation's XML file
#         objects = parse_annotation(os.path.join(voc07_path, 'Annotations', id + '.xml'))
#         if len(objects) == 0:
#             continue
#         test_objects.append(objects)
#         n_objects += len(objects)
#         test_images.append(os.path.join(voc07_path, 'JPEGImages', id + '.jpg'))

#     assert len(test_objects) == len(test_images)

#     # Save to file
#     with open(os.path.join(output_folder, 'TEST_images.json'), 'w') as j:
#         json.dump(test_images, j)
#     with open(os.path.join(output_folder, 'TEST_objects.json'), 'w') as j:
#         json.dump(test_objects, j)

#     print('\nThere are %d test images containing a total of %d objects. Files have been saved to %s.' % (
#         len(test_images), n_objects, os.path.abspath(output_folder)))

In [8]:
output_folder='../train_cdc/'
create_data_lists(output_folder)

2394 266 2394 266
['../train_cdc/train_images/201410205.jpg', '../train_cdc/train_images/2016050227.jpg'] [{'boxes': [[78, 79, 315, 239]], 'labels': [8], 'difficulties': [0]}, {'boxes': [[0, 177, 66, 291], [19, 128, 91, 246], [80, 70, 213, 288], [176, 29, 310, 221], [252, 197, 331, 299]], 'labels': [5, 5, 5, 5, 5], 'difficulties': [0, 0, 0, 0, 0]}]

There are 2394 training images containing a total of 7980 objects. Files have been saved to /home/may/Deep Learning/Object Detection/train_cdc.


In [28]:
class PascalVOCDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.
    """

    def __init__(self, data_folder, split, keep_difficult=False):
        """
        :param data_folder: folder where data files are stored
        :param split: split, one of 'TRAIN' or 'TEST'
        :param keep_difficult: keep or discard objects that are considered difficult to detect?
        """
        self.split = split.upper()

        assert self.split in {'TRAIN', 'TEST'}

        self.data_folder = data_folder
        self.keep_difficult = keep_difficult

        # Read data files
        with open(os.path.join(data_folder, self.split +'_images.json'), 'r') as j:
            self.images = json.load(j)
        with open(os.path.join(data_folder, self.split +'_objects.json'), 'r') as j:
            self.objects = json.load(j)


        assert len(self.images) == len(self.objects)

    def __getitem__(self, i):
        # Read image
        image = Image.open(self.images[i], mode='r')
        image = image.convert('RGB')

        # Read objects in this image (bounding boxes, labels, difficulties)
        objects = self.objects[i]
        boxes = torch.FloatTensor(objects['boxes'])  # (n_objects, 4)
        labels = torch.LongTensor(objects['labels'])  # (n_objects)
        difficulties = torch.ByteTensor(objects['difficulties'])  # (n_objects)

        # Discard difficult objects, if desired
        if not self.keep_difficult:
            boxes = boxes[1 - difficulties]
            labels = labels[1 - difficulties]
            difficulties = difficulties[1 - difficulties]

        # Apply transformations
        image, boxes, labels, difficulties = transform(image, boxes, labels, difficulties, split=self.split)

        return image, boxes, labels, difficulties

    def __len__(self):
        return len(self.images)

    def collate_fn(self, batch):
        """
        Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).

        This describes how to combine these tensors of different sizes. We use lists.

        Note: this need not be defined in this Class, can be standalone.

        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
        """

        images = list()
        boxes = list()
        labels = list()
        difficulties = list()

        for b in batch:
            images.append(b[0])
            boxes.append(b[1])
            labels.append(b[2])
            difficulties.append(b[3])

        images = torch.stack(images, dim=0)

        return images, boxes, labels, difficulties  # tensor (N, 3, 300, 300), 3 lists of N tensors each


In [7]:
label_map

{'aquarium': 1,
 'bottle': 2,
 'bowl': 3,
 'box': 4,
 'bucket': 5,
 'plastic_bag': 6,
 'plate': 7,
 'styrofoam': 8,
 'tire': 9,
 'toilet': 10,
 'tub': 11,
 'washing_machine': 12,
 'water_tower': 13,
 'background': 0}

In [33]:
# Data parameters
data_folder = '../train_cdc/'  # folder with data files
keep_difficult = True  # use objects considered difficult to detect?

# Model parameters
# Not too many here since the SSD300 has a very specific structure
n_classes = len(label_map)  # number of different types of objects
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Learning parameters
checkpoint = './checkpoint_ssd300-1733.pth.tar'  # path to model checkpoint, None if none
#checkpoint = None
batch_size = 8  # batch size
iterations = 500000  # number of iterations to train
workers = 4  # number of workers for loading data in the DataLoader
print_freq = 200  # print training status every __ batches
lr = 1e-3  # learning rate
decay_lr_at = [80000, 100000]  # decay learning rate after these many iterations
decay_lr_to = 0.1  # decay learning rate to this fraction of the existing learning rate
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
grad_clip = None  # clip if gradients are exploding, which may happen at larger batch sizes (sometimes at 32) - you will recognize it by a sorting error in the MuliBox loss calculation

cudnn.benchmark = True

In [34]:
from utils import *
from datasets import PascalVOCDataset
from tqdm import tqdm
from pprint import PrettyPrinter

# Good formatting when printing the APs for each class and mAP
pp = PrettyPrinter()

# # Parameters
# data_folder = '../train_cdc/'
# keep_difficult = True  # difficult ground truth objects must always be considered in mAP calculation, because these objects DO exist!
# #batch_size = 64
# workers = 4
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#checkpoint = './checkpoint_ssd300.pth.tar'

# Load model checkpoint that is to be evaluated
# checkpoint = torch.load(checkpoint)
# model = checkpoint['model']
# model = model.to(device)

# Switch to eval mode
# model.eval()

# # Load test data
# test_dataset = PascalVOCDataset(data_folder,
#                                 split='test',
#                                 keep_difficult=keep_difficult)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
#                                           collate_fn=test_dataset.collate_fn, num_workers=workers, pin_memory=True)


def evaluate(test_loader, model):
    """
    Evaluate.

    :param test_loader: DataLoader for test data
    :param model: model
    """

    # Make sure it's in eval mode
    model.eval()

    # Lists to store detected and true boxes, labels, scores
    det_boxes = list()
    det_labels = list()
    det_scores = list()
    true_boxes = list()
    true_labels = list()
    true_difficulties = list()  # it is necessary to know which objects are 'difficult', see 'calculate_mAP' in utils.py

    with torch.no_grad():
        # Batches
        for j, (images, boxes, labels, difficulties) in enumerate(tqdm(test_loader, desc='Evaluating')):
            images = images.to(device)  # (N, 3, 300, 300)

            # Forward prop.
            predicted_locs, predicted_scores = model(images)

            # Detect objects in SSD output
            det_boxes_batch, det_labels_batch, det_scores_batch = model.detect_objects(predicted_locs, predicted_scores,
                                                                                       min_score=0.01, max_overlap=0.45,
                                                                                       top_k=200)
            # Evaluation MUST be at min_score=0.01, max_overlap=0.45, top_k=200 for fair comparision with the paper's results and other repos

            # Store this batch's results for mAP calculation
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]
            difficulties = [d.to(device) for d in difficulties]

            det_boxes.extend(det_boxes_batch)
            det_labels.extend(det_labels_batch)
            det_scores.extend(det_scores_batch)
            true_boxes.extend(boxes)
            true_labels.extend(labels)
            true_difficulties.extend(difficulties)

        # Calculate mAP
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels, true_difficulties)

    # Print AP for each class
    pp.pprint(APs)

    print('\nMean Average Precision (mAP): %.3f' % mAP)
    return mAP


# if __name__ == '__main__':
#     evaluate(test_loader, model)


In [35]:
def main():
    """
    Training.
    """
    global start_epoch, label_map, epoch, checkpoint, decay_lr_at
        
    # Initialize model or load checkpoint
    if checkpoint is None:
        start_epoch = 0
        model = SSD300(n_classes=n_classes)
        # Initialize the optimizer, with twice the default learning rate for biases, as in the original Caffe repo
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = torch.optim.SGD(params=[{'params': biases, 'lr': 2 * lr}, {'params': not_biases}],
                                    lr=lr, momentum=momentum, weight_decay=weight_decay)

    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']

    # Move to default device
    model = model.to(device)
    criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)

    # Custom dataloaders
    train_dataset = PascalVOCDataset(data_folder,
                                     split='train',
                                     keep_difficult=keep_difficult)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                               collate_fn=train_dataset.collate_fn, num_workers=workers,
                                               pin_memory=True)  # note that we're passing the collate function here
    # Load test data
    test_dataset = PascalVOCDataset(data_folder,
                                    split='test',
                                    keep_difficult=keep_difficult)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                              collate_fn=test_dataset.collate_fn, num_workers=workers, pin_memory=True)
    # Good formatting when printing the APs for each class and mAP
    #pp = PrettyPrinter()
    mAP_dic = {}
    
    # Calculate total number of epochs to train and the epochs to decay learning rate at (i.e. convert iterations to epochs)
    # To convert iterations to epochs, divide iterations by the number of iterations per epoch
    # The paper trains for 120,000 iterations with a batch size of 32, decays after 80,000 and 100,000 iterations
    epochs = iterations // (len(train_dataset) // 32)
    print(len(train_dataset))
    decay_lr_at = [it // (len(train_dataset) // 32) for it in decay_lr_at]
    
    # Epochs
    for epoch in range(start_epoch, epochs):
        #print(epoch,start_epoch,epochs)

        # Decay learning rate at particular epochs
        if epoch in decay_lr_at:
            adjust_learning_rate(optimizer, decay_lr_to)

        # One epoch's training
        train(train_loader=train_loader,
              model=model,
              criterion=criterion,
              optimizer=optimizer,
              epoch=epoch)

        # Save checkpoint
        save_checkpoint(epoch, model, optimizer)
        #if epoch %3529== 0:
        if epoch %100==0:
            model.eval()
            mAP = evaluate(test_loader, model)
            mAP_dic[epoch] = mAP
            with open('./mAP_dic.pickle', 'wb') as f:
                pickle.dump(mAP_dic, f)
    return mAP_dic


def train(train_loader, model, criterion, optimizer, epoch):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()

    # Batches
    for i, (images, boxes, labels, _) in enumerate(train_loader):
        data_time.update(time.time() - start)

        # Move to default device
        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        boxes = [b.to(device) for b in boxes]
        labels = [l.to(device) for l in labels]

        # Forward prop.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, boxes, labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Clip gradients, if necessary
        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                                  batch_time=batch_time,
                                                                  data_time=data_time, loss=losses))
    del predicted_locs, predicted_scores, images, boxes, labels  # free some memory since their histories may be stored


if __name__ == '__main__':
    mAP_dic = main()
    



Loaded checkpoint from epoch 1733.

2394
Epoch: [1733][0/300]	Batch Time 0.519 (0.519)	Data Time 0.420 (0.420)	Loss 0.7604 (0.7604)	
Epoch: [1733][200/300]	Batch Time 0.094 (0.090)	Data Time 0.000 (0.003)	Loss 0.6962 (0.8879)	
Epoch: [1734][0/300]	Batch Time 0.544 (0.544)	Data Time 0.454 (0.454)	Loss 0.8015 (0.8015)	
Epoch: [1734][200/300]	Batch Time 0.088 (0.092)	Data Time 0.000 (0.003)	Loss 0.9626 (0.8898)	
Epoch: [1735][0/300]	Batch Time 0.681 (0.681)	Data Time 0.593 (0.593)	Loss 0.7828 (0.7828)	
Epoch: [1735][200/300]	Batch Time 0.088 (0.092)	Data Time 0.000 (0.003)	Loss 0.9416 (0.8618)	
Epoch: [1736][0/300]	Batch Time 0.479 (0.479)	Data Time 0.388 (0.388)	Loss 0.8860 (0.8860)	
Epoch: [1736][200/300]	Batch Time 0.090 (0.094)	Data Time 0.000 (0.005)	Loss 0.8142 (0.8522)	
Epoch: [1737][0/300]	Batch Time 0.421 (0.421)	Data Time 0.330 (0.330)	Loss 1.3307 (1.3307)	
Epoch: [1737][200/300]	Batch Time 0.089 (0.091)	Data Time 0.000 (0.002)	Loss 0.6208 (0.8840)	
Epoch: [1738][0/300]	Batch T

Epoch: [1777][0/300]	Batch Time 0.488 (0.488)	Data Time 0.395 (0.395)	Loss 1.3153 (1.3153)	
Epoch: [1777][200/300]	Batch Time 0.088 (0.093)	Data Time 0.000 (0.002)	Loss 0.4868 (0.8728)	
Epoch: [1778][0/300]	Batch Time 0.535 (0.535)	Data Time 0.445 (0.445)	Loss 1.0811 (1.0811)	
Epoch: [1778][200/300]	Batch Time 0.089 (0.092)	Data Time 0.000 (0.003)	Loss 1.2004 (0.8757)	
Epoch: [1779][0/300]	Batch Time 0.624 (0.624)	Data Time 0.535 (0.535)	Loss 0.8766 (0.8766)	
Epoch: [1779][200/300]	Batch Time 0.089 (0.094)	Data Time 0.000 (0.003)	Loss 0.7145 (0.8840)	
Epoch: [1780][0/300]	Batch Time 0.575 (0.575)	Data Time 0.481 (0.481)	Loss 0.9724 (0.9724)	
Epoch: [1780][200/300]	Batch Time 0.089 (0.092)	Data Time 0.000 (0.003)	Loss 0.9159 (0.8415)	
Epoch: [1781][0/300]	Batch Time 0.698 (0.698)	Data Time 0.613 (0.613)	Loss 1.2159 (1.2159)	
Epoch: [1781][200/300]	Batch Time 0.090 (0.094)	Data Time 0.000 (0.004)	Loss 1.0246 (0.8694)	
Epoch: [1782][0/300]	Batch Time 0.585 (0.585)	Data Time 0.493 (0.493)	

Evaluating: 100%|██████████| 34/34 [00:03<00:00, 11.12it/s]


{'aquarium': 1.0,
 'bottle': 0.1910533905029297,
 'bowl': 0.4134382903575897,
 'box': 0.48201417922973633,
 'bucket': 0.6776054501533508,
 'plastic_bag': 0.3294447958469391,
 'plate': 0.24924243986606598,
 'styrofoam': 0.6725274920463562,
 'tire': 0.7112661004066467,
 'toilet': 0.5909091234207153,
 'tub': 0.7402597069740295,
 'washing_machine': 0.5454545617103577,
 'water_tower': 0.5341055989265442}

Mean Average Precision (mAP): 0.549
Epoch: [1801][0/300]	Batch Time 0.595 (0.595)	Data Time 0.507 (0.507)	Loss 0.7972 (0.7972)	
Epoch: [1801][200/300]	Batch Time 0.087 (0.092)	Data Time 0.000 (0.003)	Loss 1.1517 (0.8803)	
Epoch: [1802][0/300]	Batch Time 0.523 (0.523)	Data Time 0.432 (0.432)	Loss 0.7936 (0.7936)	
Epoch: [1802][200/300]	Batch Time 0.095 (0.092)	Data Time 0.000 (0.003)	Loss 1.1006 (0.8445)	
Epoch: [1803][0/300]	Batch Time 0.489 (0.489)	Data Time 0.396 (0.396)	Loss 0.8365 (0.8365)	
Epoch: [1803][200/300]	Batch Time 0.091 (0.092)	Data Time 0.000 (0.002)	Loss 0.8114 (0.8489)	
Ep

Epoch: [1843][0/300]	Batch Time 0.517 (0.517)	Data Time 0.425 (0.425)	Loss 0.4513 (0.4513)	
Epoch: [1843][200/300]	Batch Time 0.091 (0.095)	Data Time 0.000 (0.007)	Loss 0.7053 (0.8707)	
Epoch: [1844][0/300]	Batch Time 0.470 (0.470)	Data Time 0.376 (0.376)	Loss 0.7663 (0.7663)	
Epoch: [1844][200/300]	Batch Time 0.088 (0.092)	Data Time 0.000 (0.002)	Loss 1.0882 (0.8744)	
Epoch: [1845][0/300]	Batch Time 0.687 (0.687)	Data Time 0.597 (0.597)	Loss 0.5199 (0.5199)	
Epoch: [1845][200/300]	Batch Time 0.088 (0.093)	Data Time 0.000 (0.003)	Loss 0.6374 (0.8516)	
Epoch: [1846][0/300]	Batch Time 0.545 (0.545)	Data Time 0.457 (0.457)	Loss 0.6691 (0.6691)	
Epoch: [1846][200/300]	Batch Time 0.094 (0.092)	Data Time 0.000 (0.003)	Loss 1.3328 (0.8484)	
Epoch: [1847][0/300]	Batch Time 0.483 (0.483)	Data Time 0.385 (0.385)	Loss 1.0741 (1.0741)	
Epoch: [1847][200/300]	Batch Time 0.091 (0.093)	Data Time 0.000 (0.004)	Loss 0.7489 (0.8705)	
Epoch: [1848][0/300]	Batch Time 0.735 (0.735)	Data Time 0.644 (0.644)	

...

Epoch: [2587][200/300]	Batch Time 0.085 (0.093)	Data Time 0.000 (0.004)	Loss 1.1876 (0.8650)	
Epoch: [2588][0/300]	Batch Time 0.600 (0.600)	Data Time 0.508 (0.508)	Loss 1.1638 (1.1638)	
Epoch: [2588][200/300]	Batch Time 0.097 (0.093)	Data Time 0.000 (0.003)	Loss 0.6664 (0.8920)	
Epoch: [2589][0/300]	Batch Time 0.460 (0.460)	Data Time 0.368 (0.368)	Loss 0.4852 (0.4852)	
Epoch: [2589][200/300]	Batch Time 0.089 (0.092)	Data Time 0.000 (0.003)	Loss 0.6215 (0.8654)	
Epoch: [2590][0/300]	Batch Time 0.468 (0.468)	Data Time 0.373 (0.373)	Loss 0.5907 (0.5907)	
Epoch: [2590][200/300]	Batch Time 0.089 (0.092)	Data Time 0.000 (0.002)	Loss 0.5348 (0.8382)	
Epoch: [2591][0/300]	Batch Time 0.502 (0.502)	Data Time 0.404 (0.404)	Loss 1.4411 (1.4411)	
Epoch: [2591][200/300]	Batch Time 0.097 (0.092)	Data Time 0.000 (0.002)	Loss 1.0278 (0.8574)	
Epoch: [2592][0/300]	Batch Time 0.549 (0.549)	Data Time 0.457 (0.457)	Loss 0.9847 (0.9847)	
Epoch: [2592][200/300]	Batch Time 0.091 (0.092)	Data Time 0.000 (0.002

Evaluating: 100%|██████████| 34/34 [00:03<00:00, 11.32it/s]


{'aquarium': 1.0,
 'bottle': 0.17439758777618408,
 'bowl': 0.41288870573043823,
 'box': 0.4837130904197693,
 'bucket': 0.6798797845840454,
 'plastic_bag': 0.3164287805557251,
 'plate': 0.2962879240512848,
 'styrofoam': 0.6642807126045227,
 'tire': 0.7074383497238159,
 'toilet': 0.5909091234207153,
 'tub': 0.7402597069740295,
 'washing_machine': 0.5454545617103577,
 'water_tower': 0.5405352711677551}

Mean Average Precision (mAP): 0.550
Epoch: [2601][0/300]	Batch Time 0.503 (0.503)	Data Time 0.412 (0.412)	Loss 0.6933 (0.6933)	
Epoch: [2601][200/300]	Batch Time 0.088 (0.091)	Data Time 0.000 (0.002)	Loss 0.6064 (0.8618)	
Epoch: [2602][0/300]	Batch Time 0.506 (0.506)	Data Time 0.407 (0.407)	Loss 0.9629 (0.9629)	
Epoch: [2602][200/300]	Batch Time 0.090 (0.093)	Data Time 0.000 (0.003)	Loss 0.4602 (0.8527)	
Epoch: [2603][0/300]	Batch Time 0.639 (0.639)	Data Time 0.554 (0.554)	Loss 0.9911 (0.9911)	
Epoch: [2603][200/300]	Batch Time 0.090 (0.093)	Data Time 0.000 (0.003)	Loss 0.6399 (0.8762)	
Ep

Epoch: [2643][0/300]	Batch Time 0.536 (0.536)	Data Time 0.439 (0.439)	Loss 0.5449 (0.5449)	
Epoch: [2643][200/300]	Batch Time 0.093 (0.093)	Data Time 0.000 (0.003)	Loss 0.6998 (0.8631)	
Epoch: [2644][0/300]	Batch Time 0.428 (0.428)	Data Time 0.337 (0.337)	Loss 0.6632 (0.6632)	
Epoch: [2644][200/300]	Batch Time 0.089 (0.092)	Data Time 0.000 (0.003)	Loss 0.7721 (0.8779)	
Epoch: [2645][0/300]	Batch Time 0.453 (0.453)	Data Time 0.359 (0.359)	Loss 1.2172 (1.2172)	
Epoch: [2645][200/300]	Batch Time 0.087 (0.091)	Data Time 0.000 (0.002)	Loss 0.8311 (0.8680)	
Epoch: [2646][0/300]	Batch Time 0.614 (0.614)	Data Time 0.522 (0.522)	Loss 1.0172 (1.0172)	
Epoch: [2646][200/300]	Batch Time 0.088 (0.091)	Data Time 0.000 (0.003)	Loss 0.6548 (0.8953)	
Epoch: [2647][0/300]	Batch Time 0.508 (0.508)	Data Time 0.412 (0.412)	Loss 0.7390 (0.7390)	
Epoch: [2647][200/300]	Batch Time 0.090 (0.092)	Data Time 0.000 (0.002)	Loss 0.8761 (0.8412)	
Epoch: [2648][0/300]	Batch Time 0.607 (0.607)	Data Time 0.516 (0.516)	

KeyboardInterrupt: 

In [36]:
from utils import *
from datasets import PascalVOCDataset
from tqdm import tqdm
from pprint import PrettyPrinter

# Good formatting when printing the APs for each class and mAP
pp = PrettyPrinter()

# Parameters
data_folder = '../train_cdc/'
keep_difficult = True  # difficult ground truth objects must always be considered in mAP calculation, because these objects DO exist!
batch_size = 64
workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = './checkpoint_ssd300.pth.tar'

# Load model checkpoint that is to be evaluated
checkpoint = torch.load(checkpoint)
model = checkpoint['model']
model = model.to(device)

# Switch to eval mode
model.eval()

# Load test data
test_dataset = PascalVOCDataset(data_folder,
                                split='test',
                                keep_difficult=keep_difficult)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                          collate_fn=test_dataset.collate_fn, num_workers=workers, pin_memory=True)


def evaluate(test_loader, model):
    """
    Evaluate.

    :param test_loader: DataLoader for test data
    :param model: model
    """

    # Make sure it's in eval mode
    model.eval()

    # Lists to store detected and true boxes, labels, scores
    det_boxes = list()
    det_labels = list()
    det_scores = list()
    true_boxes = list()
    true_labels = list()
    true_difficulties = list()  # it is necessary to know which objects are 'difficult', see 'calculate_mAP' in utils.py

    with torch.no_grad():
        # Batches
        for i, (images, boxes, labels, difficulties) in enumerate(tqdm(test_loader, desc='Evaluating')):
            images = images.to(device)  # (N, 3, 300, 300)

            # Forward prop.
            predicted_locs, predicted_scores = model(images)

            # Detect objects in SSD output
            det_boxes_batch, det_labels_batch, det_scores_batch = model.detect_objects(predicted_locs, predicted_scores,
                                                                                       min_score=0.01, max_overlap=0.45,
                                                                                       top_k=200)
            # Evaluation MUST be at min_score=0.01, max_overlap=0.45, top_k=200 for fair comparision with the paper's results and other repos

            # Store this batch's results for mAP calculation
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]
            difficulties = [d.to(device) for d in difficulties]

            det_boxes.extend(det_boxes_batch)
            det_labels.extend(det_labels_batch)
            det_scores.extend(det_scores_batch)
            true_boxes.extend(boxes)
            true_labels.extend(labels)
            true_difficulties.extend(difficulties)
            #print(det_boxes,true_boxes)

        # Calculate mAP
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels, true_difficulties)

    # Print AP for each class
    pp.pprint(APs)

    print('\nMean Average Precision (mAP): %.3f' % mAP)


if __name__ == '__main__':
    evaluate(test_loader, model)


Evaluating: 100%|██████████| 5/5 [00:03<00:00,  1.65it/s]


{'aquarium': 1.0,
 'bottle': 0.18416032195091248,
 'bowl': 0.40794846415519714,
 'box': 0.4847709536552429,
 'bucket': 0.6801402568817139,
 'plastic_bag': 0.318746417760849,
 'plate': 0.28386861085891724,
 'styrofoam': 0.665507972240448,
 'tire': 0.7103840708732605,
 'toilet': 0.5909091234207153,
 'tub': 0.7402597069740295,
 'washing_machine': 0.5454545617103577,
 'water_tower': 0.5279167294502258}

Mean Average Precision (mAP): 0.549
